In [82]:
import numpy as np
import matplotlib.pyplot
from timeit import default_timer as timer
import scipy as sp

The goal of this notebook is to attempt to implement and recreate the results of E.H. Clausen et. al (Phys. Rev. A 105, 063709 – Published 9 June 2022). I will be focusing on the molecular transition example in that paper.

In [83]:
#Natural constants
c = 299792458

#Constants of the experiment
transition_wavelength = 6.17*1e-6 #6.17um
transition_frequency = 2*np.pi*c/transition_wavelength #
laser_width = 0.5*1e9 #0.5 GHz
oop_frequency = 2*np.pi*295.7*1e3
ip_freqyency = 2*np.pi*162*1e3

LDP_ip_t = 0.0136
LDP_op_t = 0.0159

heating_rate_ip = 14
heating_rate_op = 1.7

saturation  = 4*1e3
gamma_transition = 2*np.pi*2.5

#Define spontaneous decay emission pattern
sp_dec_emission_pattern = 1/(4*np.pi)

#Sidebands and max n
max_s_op = 1
max_s_ip = 1

max_n_ip = 2
max_n_op = 2

Format populations into two arrays/matrices. One contains probabilities for excited state target ion, and one contains ground state target ion. Let the indexes (m,n) denote n_ip, n_op populations, i.e.

$M_{exc}= \begin{bmatrix}
P_{e,00} & P_{e,01}\\
P_{e,01} & P_{e,11}
\end{bmatrix}$
and so forth

In [84]:
M_exc_init = np.zeros((max_n_ip+1,max_n_op+1))
M_gs_init = np.zeros((max_n_ip+1,max_n_op+1))
M_gs_init[0,0] = 1

print(M_exc_init[2,:])

[0. 0. 0.]


In [85]:
def heatingGain(heating_rate_ip,heating_rate_op,Pmatrix,n_ip,n_op):
    #Calculate the population INCREASE in a given mode
    #PARAMETERS:
    # heating_rate_ip: heating rate of the in-phase mode
    # heating_rate_op: heating rate of the out-of-phase mode
    # Pmatrix: probability matrix on which heating is occuring, i.e. the excited or ground state probability matrix
    # n_ip: in-phase fock state on which heating is calculated
    # n_op: out-of-phase fock state on which heating is calculated
    # 
    #  
    if n_ip == 0:
        ip_heating_gain = 0
    else:
        ip_heating_gain = heating_rate_ip*Pmatrix[n_ip-1,n_op]
    if n_op ==0:
        op_heating_gain = 0
    else:
        op_heating_gain = heating_rate_op*Pmatrix[n_ip,n_op-1]
    return ip_heating_gain + op_heating_gain

def heatingLoss(heating_rate_ip,heating_rate_op,Pmatrix,n_ip,n_op):
    #Calculate the population DECREASE due to heating in a given mode
    #PARAMETERS:
    # heating_rate_ip: heating rate of the in-phase mode
    # heating_rate_op: heating rate of the out-of-phase mode
    # Pmatrix: probability matrix on which heating is occuring, i.e. the excited or ground state probability matrix
    # n_ip: in-phase fock state on which heating is calculated
    # n_op: out-of-phase fock state on which heating is calculated

    loss = -(heating_rate_op+heating_rate_ip)*Pmatrix[n_ip,n_op]
    return loss

def absorptionLoss(LDP_ip_t,LDP_op_t,gamma_transition,saturation,max_s_ip,max_s_op,Pmatrix,n_ip,n_op):
    #PARAMETERS:
    # LDP_ip_t: LDP on the 1st in-phase sideband on target ion.
    # LDP_op_t: LDP on the 1st out-of-phase sideband on target ion
    # gamma_transition: lifetime of transition (2pi x Hz)
    # saturation: saturation parameter of the laser, s = I/I_sat
    # Pmatrix: probability matrix on which heating is occuring, i.e. the excited or ground state probability matrix
    # n_ip: in-phase fock state on which heating is calculated
    # n_op: out-of-phase fock state on which heating is calculated

    absorptionrate = gamma_transition*saturation
    sum = 0
    for s_ip in np.arange(-max_s_ip,max_s_ip+1):
        for s_op in np.arange(-max_s_op,max_s_op+1):
            print('------------------------------------')
            print('Entering s_ip,s_op = ',s_ip, s_op)
            print('n_ip,n_op = ', n_ip, n_op)
            sum -= absorptionrate*Pmatrix[n_ip,n_op]*(np.abs(chi(LDP_ip_t,LDP_op_t,n_ip,n_op,s_ip,s_op)))**2
            print('Exited ok')
    return sum

def chi(LDP_ip_t,LDP_op_t,n_ip,n_op,s_ip,s_op):
    #Returns the higher-order LDP for absorption on the s_ip,s_op'th sideband
    if (n_ip+s_ip >max_n_ip) or (n_op+s_op>max_n_op) or (np.sign(n_ip+s_ip) == -1) or (np.sign(n_op+s_op) == -1):
        return 0
    else:
        factor1 = LDP_ip_t**np.abs(s_ip)/(sp.special.factorial(np.abs(s_ip))) * np.sqrt(sp.special.factorial(np.max([n_ip,n_ip+s_ip]))/(sp.special.factorial(np.min([n_ip,n_ip+s_ip]))))
        factor2 = LDP_op_t**np.abs(s_op)/(sp.special.factorial(np.abs(s_op))) * np.sqrt(sp.special.factorial(np.max([n_op,n_op+s_op]))/(sp.special.factorial(np.min([n_op,n_op+s_op]))))
        return factor1*factor2
    

def dM_gs(max_n_ip,max_n_op,max_s_ip,max_s_op,LDP_ip_t,LDP_op_t,gamma_transition,saturation,heating_rate_ip,heating_rate_op,M_gs,M_exc):
    dM = np.zeros((max_n_ip+1,max_n_op+1))
    for n_ip in range(max_n_ip+1):
        for n_op in range(max_n_op+1):
            heating_increase = 0#heatingGain(heating_rate_ip,heating_rate_op,M_exc,n_ip,n_op)
            heating_decrease = 0#heatingLoss(heating_rate_ip,heating_rate_op,M_exc,n_ip,n_op)
            absorption_decrease = absorptionLoss(LDP_ip_t,LDP_op_t,gamma_transition,saturation,max_s_ip,max_s_op,M_gs,n_ip,n_op)
            dM[n_ip,n_op] += heating_increase+heating_decrease+absorption_decrease
            print('Exited succesfully at ', n_ip, '.', n_op)
    return dM




t0 = timer()
print(dM_gs(max_n_ip,max_n_op,max_s_ip,max_s_op,LDP_ip_t,LDP_op_t,gamma_transition,saturation,heating_rate_ip,heating_rate_op,M_gs_init,M_exc_init))
t_end = timer()
print('Calculating and printing dM took ', t_end-t0)

------------------------------------
Entering s_ip,s_op =  -1 -1
n_ip,n_op =  0 0
Exited ok
------------------------------------
Entering s_ip,s_op =  -1 0
n_ip,n_op =  0 0
Exited ok
------------------------------------
Entering s_ip,s_op =  -1 1
n_ip,n_op =  0 0
Exited ok
------------------------------------
Entering s_ip,s_op =  0 -1
n_ip,n_op =  0 0
Exited ok
------------------------------------
Entering s_ip,s_op =  0 0
n_ip,n_op =  0 0
Exited ok
------------------------------------
Entering s_ip,s_op =  0 1
n_ip,n_op =  0 0
Exited ok
------------------------------------
Entering s_ip,s_op =  1 -1
n_ip,n_op =  0 0
Exited ok
------------------------------------
Entering s_ip,s_op =  1 0
n_ip,n_op =  0 0
Exited ok
------------------------------------
Entering s_ip,s_op =  1 1
n_ip,n_op =  0 0
Exited ok
Exited succesfully at  0 . 0
------------------------------------
Entering s_ip,s_op =  -1 -1
n_ip,n_op =  0 1
Exited ok
------------------------------------
Entering s_ip,s_op =  -1 0